In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
workout_data = pd.read_csv('../data/current/workout_data.csv')
workout_data

In [ ]:
enriched_workout_data = pd.read_csv('../data/current/enriched_workout_data.csv')
enriched_workout_data

In [ ]:
workout_day_exercises = pd.read_csv('../data/current/workout_day_exercises.csv')
workout_day_exercises

In [ ]:
workout_days = pd.read_csv('../data/current/workout_days.csv')
workout_days

In [ ]:
workout_exercises = pd.read_csv('../data/current/workout_exercises.csv')
workout_exercises

# Distributions of workout_exercises columns

In [ ]:
# Distributions of columns
for column in workout_exercises.drop(columns=["Desc", "Title"]).columns:
    fig = px.histogram(workout_exercises, x=column)
    fig.show()

In [ ]:
# Delete column 'RatingDesc'
workout_exercises = workout_exercises.drop(columns=['RatingDesc'])

# Analytics

In [ ]:
# Metrics of workout exercises
# Favorite Exercise Type
print(workout_exercises['Type'].value_counts().idxmax())
# Favorite Equipment
print(workout_exercises['Equipment'].value_counts().idxmax())
# Favorite Exercise Level
print(workout_exercises['Level'].value_counts().idxmax())
# Average Rating of Exercises
print(workout_exercises['Rating'].mean().round(2))

In [ ]:
# Calculate histogram data
hist_data = enriched_workout_data['BodyPart'].value_counts().sort_values(ascending=False)

# Create figure
fig = go.Figure()

# Histogram of Targeted Muscle Group
fig.add_trace(go.Bar(
    x=hist_data.index,
    y=hist_data.values,
    marker_color="#4a3e80",
    hovertemplate="Number of Sets: %{y}<extra></extra>",
))

# Update layout
fig.update_layout(
    title="Histogram of Targeted Muscle Group",
    xaxis_title="Muscle Group",
    yaxis_title="Number of Sets",
    hovermode="x unified",
    font=dict(
        family="Roboto, sans-serif",
        size=18,
        color="#333"
    ),
    template="plotly_white",
    height=700,
    plot_bgcolor="#f4f4f4",
    paper_bgcolor="#f4f4f4",
    yaxis=dict(
        gridcolor="#ddd",
        zerolinecolor="#ddd",
    ),
)

fig.show()

In [ ]:
# Count of workout types
workout_occurrences = workout_days["WORKOUT"].value_counts()

# Workout types above 5 occurrences
valid_workouts = workout_occurrences[workout_occurrences > 5].index
valid_workouts_data = workout_days[workout_days['WORKOUT'].isin(valid_workouts)]

# Donut chart of workout types (Workouts above 5 occurrences)
fig = go.Figure()

# Create donut chart
fig.add_trace(go.Pie(
    labels=valid_workouts_data['WORKOUT'].value_counts().index,
    values=valid_workouts_data['WORKOUT'].value_counts().values,
    hole=0.6,
    marker_colors=px.colors.qualitative.Bold,
    hoverinfo="label+percent",
    hovertemplate="Workout: %{label}<br>Percentage: %{percent}<br>Occurrences: %{value}<extra></extra>",
    textinfo="value",
    textposition="inside"
))

# Update layout
fig.update_layout(
    title="Donut Chart of Workout Types",
    font=dict(
        family="Roboto, sans-serif",
        size=18,
        color="#7f7f7f"
    ),
    template="plotly_white",
    height=500,
)

fig.show()

In [ ]:
# Get top 5 exercises
top_5_exercises = workout_day_exercises['EXERCISE'].value_counts().head(5).index
top_5_exercises_data = workout_day_exercises[workout_day_exercises['EXERCISE'].isin(top_5_exercises)]
top_5_exercises_data.loc[:, 'DATE'] = pd.to_datetime(top_5_exercises_data['DATE'])

# Define color map for the exercises
color_map = dict(zip(top_5_exercises, px.colors.qualitative.Bold))

# Create figure
fig = go.Figure()

# Add a scatter trace for each exercise
for exo in top_5_exercises:
    exo_data = top_5_exercises_data[top_5_exercises_data['EXERCISE'] == exo]
    fig.add_trace(
        go.Scatter(
            x=exo_data['DATE'],
            y=exo_data['AVERAGE_WEIGHT'],
            mode='markers',
            marker=dict(size=exo_data['AVERAGE_REPS'], color=color_map[exo]),
            name=exo,
            hovertext=exo_data['EXERCISE'],
            hovertemplate=
            "<b>%{hovertext}</b><br>" +
            "Average Weight: %{y:.1f} kg<br>" +
            "Average Reps: %{marker.size:.1f}<extra></extra>",
        )
    )
    
# Update layout
fig.update_layout(
    title="Evolution of Weights and Reps over Time (Top 5 Exercises)",
    xaxis_title=None,
    yaxis_title="Average Weight",
    hovermode="x unified",
    font=dict(
        family="Roboto, sans-serif",
        size=18,
        color="#7f7f7f"
    ),
    template="plotly_white",
    height=700,
)

fig.show()